In [1]:
import sys
sys.path.append("/home/vatalla/work")
#import networkx as nx
import pandas as pd
import numpy as np
import qgrid
from IPython.display import  HTML, Image
import matplotlib.pyplot as plt
from mycluster.impala_query import Impala
from collections import namedtuple
from scipy.special import gammaln


%matplotlib inline

In [2]:
FIRMA = "100"
LAND = "004"
WG = "666"
FROM_DATE = "2017-04-24"
TO_DATE = "2018-04-24"

### Aggregate Data  take only 3000 

In [3]:
query_template= """
SELECT r.varianten_nummer, 
       sum(r.bestell_stueckzahl) as n_ordered, 
       sum(r.retoure_stueckzahl) as n_returned, 
       sum(r.retoure_stueckzahl_zu_gross) as n_returned_too_large, 
       sum(r.retoure_stueckzahl_zu_klein) as n_returned_too_small,
       sum(r.retoure_stueckzahl) / sum(r.bestell_stueckzahl) as return_ratio
From
(Select 
       konto_nummer,
       auftrags_identifikation,
       varianten_groesse,
       grund_retoure,
       waren_gruppe,
       varianten_nummer,
       retoure_stueckzahl,
       bestell_stueckzahl,
       CASE WHEN grund_retoure in ('51','53','55','61','63','65') then retoure_stueckzahl else 0 end as retoure_stueckzahl_zu_klein,
       CASE WHEN grund_retoure in ('50','52','54','60','62','64') then retoure_stueckzahl else 0 end as retoure_stueckzahl_zu_gross
FROM auftragsdaten.erfolge_a
WHERE datum_auftrag BETWEEN '{from_date}' AND '{to_date}'
and firma = '{firma}'
AND land = '{land}'
and waren_gruppe = '{wg}') r
GROUP BY r.varianten_nummer
"""

In [4]:
# TODO: remove limit 1000  due to memory error
query = query_template.format(firma=FIRMA, land=LAND, wg=WG, from_date=FROM_DATE, to_date=TO_DATE) + "limit 3000"
data = Impala("default").query(query).set_index('varianten_nummer')

In [10]:
data.to_csv("/home/vatalla/Desktop/illustration_return_bayes.csv", index=False)

In [11]:
from matplotlib import rc, rcParams

rc('text', usetex=True)
#color="yellow"
color='#00aac2'
img_format = "eps"
rcParams.update({'figure.figsize': [14,5],
                 'text.color' : color,
                 'font.size': 20,
                 'axes.labelcolor' : color, 
                 'xtick.color': color, 
                 'ytick.color': color,
                 'legend.framealpha': 0, 
                 'legend.edgecolor': 'none'
                 })


### Make some plots for illustration

In [12]:
def marginal_posterior(A, B, n, y):
    """compute the marginal posterior of alpha and beta in the hierarchical model in a grid
    This is done on the log scale and then back transformed to avoid numerical underflow
    A: A_grid
    B: B_grid
    n: array of tries
    y: array of successes
    """
    lp = (
      - 5/2 * np.log(A + B[:, None])
      + np.sum(
            gammaln(A + B[:, None])
          - gammaln(A)
          - gammaln(B[:, None])
          + gammaln(A + y[:, None, None])
          + gammaln(B[:,None] + (n - y)[:, None, None])
          - gammaln(A + B[:,None] + n[:, None, None]),
            axis=0
        )
    )
    lp -= lp.max()
    return np.exp(lp)

def sample_alpha_beta_posterior(A, B, p, nsamp=1000):
    samp_indices = np.unravel_index(
        np.random.choice(p.size, size=nsamp, p=p.ravel()/p.sum()),
        p.shape
    )
    samp_A = A[samp_indices[1]]
    samp_B = B[samp_indices[0]]
    # add random jitter, see BDA3 p. 76
    samp_A += (np.random.rand(nsamp) - 0.5) * (A[1]-A[0])
    samp_B += (np.random.rand(nsamp) - 0.5) * (B[1]-B[0])
    return pd.DataFrame({'alpha':samp_A, 'beta':samp_B})

def get_predictive_alpha_beta_trace_rq(data:pd.DataFrame, trial: str='n_ordered', success: str='n_returned'):
    A = np.linspace(0.1, 20, 500)
    B = np.linspace(10, 35, 500)
    ns = data[trial].values
    ys = data[success].values
    return sample_alpha_beta_posterior(A, B, marginal_posterior(A, B, ns, ys))
  
def get_posterior(alpha_beta_trace, trial, success):
    return np.random.beta(alpha_beta_trace.alpha + success, alpha_beta_trace.beta + (trial - success))

In [13]:
def plot_return_problem(ax, prior, posterior, buys, returns):
    #fig, ax = plt.subplots(figsize = (12, 4))
    mean = posterior.mean()
    rq=returns/buys
    ax.hist(prior, color=color, bins=20, normed=True, label='prior');
    ax.hist(posterior, color="#F08080", bins=20, normed=True, label='posterior');
    ax.set_xlim(0,1)
    ax.set_ylim(0,8)
    ax.axvline(prior.mean(), linestyle='--', color='black', alpha=.3)

    ax.annotate(r"$\langle\theta\rangle$",
            bbox=dict(boxstyle="round4", fc="w"),
                color='k',
            xy=(mean, 0), xycoords='data',
            xytext=(mean, 4), textcoords='data',size=15, va="center", ha="center",
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )
    
    
    ax.annotate("rq",
            bbox=dict(boxstyle="round4", fc="w"),
            xy=(rq, 0), xycoords='data',color='k',
            xytext=(rq, 1.5), textcoords='data',size=15, va="center", ha="center",
            arrowprops=dict(arrowstyle="->",
                            connectionstyle="arc3"),
            )
    low_percentile = np.percentile(posterior, 5)
    high_percentile = np.percentile(posterior, 95)
    ax.annotate("",
            xy=(low_percentile, 7), xycoords='data',
            xytext=(high_percentile, 7), textcoords='data',
            arrowprops=dict(arrowstyle="<->", linestyle='-', color=color),
            )
    spread_text_x = .5 * (low_percentile + high_percentile)
    
    ax.annotate(
    'Spread', xy=(spread_text_x, 6), xycoords='data', ha="center", size=15,
    xytext=(spread_text_x, 6), textcoords='data')
    
    ax.set_ylabel(r"$p(\theta)$")
    ax.set_yticklabels([])
    ax.set_yticks([])
    ax.set_title("{r} returned / {b} ordered".format(r=returns, b=buys))
    ax.legend()
    
    ax.spines['bottom'].set_color(color)
    ax.spines['top'].set_color(color) 
    ax.spines['right'].set_color(color)
    ax.spines['left'].set_color(color)
    
    return ax

In [14]:
setup = namedtuple('setup', ['prior', 'posterior', 'buys', 'returns'])

In [ ]:
alpha_beta_trace_rq = get_predictive_alpha_beta_trace_rq(data)
prior = get_posterior(alpha_beta_trace_rq, 0 , 0)
setups = [setup(prior, get_posterior(alpha_beta_trace_rq, 10, 8), 10, 8),
          setup(prior, get_posterior(alpha_beta_trace_rq, 50, 40), 50, 40),
          setup(prior, get_posterior(alpha_beta_trace_rq, 100, 80), 100, 80)#,
          #setup(prior, get_posterior(alpha_beta_trace_rq, 1000, 800), 1000, 800)
          
         ]

In [ ]:
fig, ax = plt.subplots(nrows = 3, figsize = (12, 8), sharex=True, sharey=True)
for i, s in enumerate(setups):
    ax[i] = plot_return_problem(ax[i], s.prior, s.posterior, s.buys, s.returns)
    ax[i].patch.set_visible(False)
ax[2].set_xlabel(r"$\theta$")
fig.patch.set_alpha(0.0) # transparent background
fig.savefig("img/return_machinery." + img_format, format=img_format, bbox_inches=0, tranparent=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.hist(data.return_ratio, color="#F08080", bins=20, label='Distribution of return ratios', normed=True);
ax.hist(prior, color=color, bins=20, alpha=.7, label='Predictive Prior',  normed=True)
ax.set_xlabel(r"\theta")
ax.set_ylabel(r"$p(\theta)$")
ax.set_yticklabels([])
ax.set_yticks([])
ax.legend()
ax.spines['bottom'].set_color(color)
ax.spines['left'].set_color(color)

for side in ['right','top']:
    ax.spines[side].set_visible(False)
    
ax.patch.set_visible(False)
ax.legend()
fig.patch.set_alpha(0.0) # transparent background

fig.savefig("img/return_problem." + img_format, format=img_format, bbox_inches=0, tranparent=True)

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 4))
ax.hist(data.return_ratio.values, color="#F08080", bins=20, normed=True);
ax.set_xlabel(r"Return ratio")
ax.set_ylabel(r"$p$")
ax.set_yticklabels([])
ax.set_yticks([])
ax.legend()
ax.spines['bottom'].set_color(color)
ax.spines['left'].set_color(color)

for side in ['right','top']:
    ax.spines[side].set_visible(False)
    
ax.patch.set_visible(False)
ax.legend()
fig.patch.set_alpha(0.0) # transparent background

fig.savefig("img/motivation." + img_format, format=img_format, bbox_inches=0, tranparent=True)